In [1]:
import os
import sys
sys.path.append('../../')
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import shap
import lightgbm

import recs.path as path
from recs.clf import Classifier
from recs.backtest import get_backtest_answer, model_score_logger
from recs.util import readjson2dict, id2cat, save_model, save2json, recall_evaluate
from sklearn.model_selection import train_test_split

print(lightgbm.__version__)
print(shap.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

3.1.1
0.39.0


In [2]:
params = {
    "best_sell_n": 53,
    "n_estimators":250,
    "max_depth":3,
    "negative_sampling":True,
    "train_period": [201600, 202000],
    "best_sell_period":[201800 ,202000],
    "predict_period": [201912, 202007],
    "input_time": 201912
}

In [3]:
u2idx = readjson2dict("crm_idx")
i2idx = readjson2dict("fund_info_idx")

if params["negative_sampling"]:
    read_path = os.path.join(path.data_path, "merge_ns_diff.pkl")
else:
    read_path = os.path.join(path.data_path, "merge_diff.pkl")

trans_buy_path = os.path.join(path.data_path, "trans_buy.pkl")

trans_buy = pd.read_pickle(trans_buy_path)
merge = pd.read_pickle(read_path)
merge = merge.replace(float('-inf'), 0)
print(merge.y.value_counts())

merge= merge[
    (merge.yyyymm >= params['train_period'][0]) & 
    (merge.yyyymm <= params['train_period'][1])
]

print(f"merge data length: {len(merge)} ")
print(merge.y.value_counts())
merge

0    621392
1    267184
Name: y, dtype: int64
merge data length: 606357 
0    417746
1    188611
Name: y, dtype: int64


,id_number,fund_id,yyyymm,y,local_foreign_total,local_total,local_demand_deposit,local_fixed_deposit,foreign_total,foreign_fixed_deposit,...,currency_4,currency_5,currency_6,currency_7,currency_8,currency_9,currency_10,currency_11,currency_12,currency_13
2,13651,748,201906,1,16.179238,16.069354,16.069354,0.00000,13.916462,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,13651,2053,201708,1,15.772270,15.712601,15.712601,0.00000,12.923644,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,13651,2339,201912,1,15.137757,13.303332,13.303332,0.00000,14.963754,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,13653,2518,201705,1,14.591028,12.056824,12.056824,0.00000,14.508380,12.356740,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,13654,2091,201602,1,9.755046,9.232395,9.232395,0.00000,8.856234,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074543,67343,3133,201903,0,14.981091,14.782627,11.224203,14.75373,13.266349,13.258908,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1074544,67343,2301,201903,0,14.981091,14.782627,11.224203,14.75373,13.266349,13.258908,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1074545,67343,3184,201903,0,14.981091,14.782627,11.224203,14.75373,13.266349,13.258908,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1074546,67343,3242,201903,0,14.981091,14.782627,11.224203,14.75373,13.266349,13.258908,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
# merge = merge.drop(columns=['diff_month', 'diff_quarter', 'diff_half_year', 'diff_year', 'diff_2year'])
x = merge.drop(columns=['y', 'id_number', 'fund_id'])
y = merge['y']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
c = Classifier(x_train, x_test, y_train, y_test)

# c.do_gradient_boosting(
#     n_estimators=params["n_estimators"], 
#     max_depth=params["max_depth"]
# )

# c.do_lgbm(
#     n_estimators=params["n_estimators"], 
#     max_depth=params["max_depth"]
# )
# c.do_bagginregression()

# c.do_adaboost(
#     n_estimators=params["n_estimators"], 
# )

c.do_random_forest(
    n_estimators=params["n_estimators"], 
    max_depth=params["max_depth"]
)

# c.do_logistic()
# c.do_xgboost()


Training RandomForestClassifier


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:  1.5min finished


In [6]:
c.check_score()
c.score_result

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    5.2s finished
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


{'model_score': 0.6888256697279859,
 'accuracy_score': 0.6894171779141104,
 'f1_score': 0.0,
 'precision_score': 0.0,
 'recall_score': 0.0}

In [7]:
# inference input data
equity_path = os.path.join(path.data_path, "equity.pkl")
crm_path = os.path.join(path.data_path, "crm_diff.pkl")

crm = pd.read_pickle(crm_path)
crm = crm.replace(float('-inf'), 0)

equity = pd.read_pickle(equity_path)
i2idx = readjson2dict('fund_info_idx')


In [8]:
best_sell_n = list(trans_buy[
    (trans_buy.buy_date > params["best_sell_period"][0]*100) & 
    (trans_buy.buy_date < params["best_sell_period"][1]*100)
].fund_id.value_counts().index)[:params["best_sell_n"]]

In [9]:

targets = crm[crm.yyyymm == params["input_time"]]
funds = equity[    
    (equity.yyyymm == params["input_time"]) &
    (equity.fund_id.isin(best_sell_n))
]
targets['k'] = 0
funds['k'] = 0
print(f'{len(targets)} users, {len(funds)} items')
pred_input = targets.merge(funds, how='outer')
# pred_input.columns


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


69839 users, 53 items


In [10]:
test = get_backtest_answer(trans_buy, targets['id_number'].tolist(), 
                           params["predict_period"][0]*100, 
                           params["predict_period"][1]*100)

100%|██████████| 13885/13885 [00:38<00:00, 360.10it/s]


In [11]:
x_predict = pred_input[x_test.columns]
x_user_fund = pred_input[['id_number', 'fund_id']]
# Predict
predict_prob = c.model.predict_proba(x_predict)
pred_y = c.model.predict(x_predict)
print(f"Total number of forecasts: {len(predict_prob)}.")
print("Prediction y Probability:")
print(predict_prob)
print("Prediction y:")
print(pred_y)
print("Prediction y Counts:")
print(np.unique(pred_y, return_counts=True))
x_user_fund['prob'] = predict_prob[:, 1]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:   34.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Total number of forecasts: 3701467.
Prediction y Probability:
[[0.74644534 0.25355466]
 [0.74389231 0.25610769]
 [0.68957794 0.31042206]
 ...
 [0.65344633 0.34655367]
 [0.64991328 0.35008672]
 [0.73701048 0.26298952]]
Prediction y:
[0 0 0 ... 0 0 0]
Prediction y Counts:
(array([0]), array([3701467]))


[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:   34.4s finished

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
# x_user_fund[x_user_fund['id_number'] == 65136].sort_values(by=['prob'], ascending=False)[:10]

In [13]:
rec_result = {}

# sort_values to speed up
x_user_fund.sort_values(by=['prob'], ascending=False, inplace=True)

for user in tqdm(x_user_fund['id_number'].unique()):
    ignore_item = trans_buy[
        (trans_buy['id_number'] == user) &
        (trans_buy.buy_date >= params['train_period'][0]*100) & 
        (trans_buy.buy_date <= params['train_period'][1]*100)
    ]['fund_id'].unique().tolist()
    
    tmp = x_user_fund[x_user_fund['id_number'] == user]
    
    rec_fund = tmp[~tmp['fund_id'].isin(ignore_item)].head(10)['fund_id'].tolist()
    
    rec_result[user] = rec_fund


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 69839/69839 [05:40<00:00, 204.95it/s]


In [14]:
print(c.model_name)
scores = recall_evaluate(rec_result, test)
print(scores)
model_score_logger(c.model_name, params, scores)

RandomForestClassifier


100%|██████████| 13885/13885 [00:00<00:00, 131828.28it/s]

{'recall@3': 0.023159111087224217, 'recall@5': 0.03846192220404247, 'recall@10': 0.06756645637110932, 'recall@all': 0.06756645637110932}


In [15]:
# explainer = shap.Explainer(c.model)
# shap_vals = explainer(x_test)

In [16]:
# shap.initjs()
# shap.plots.force(shap_vals[1])

In [17]:
# shap.plots.bar(shap_vals[456], max_display=15)

In [18]:
# shap.plots.bar(shap_vals, max_display=30)